In [ ]:
test_dataset = CIFAR10_Lab(train=False)  # Указываем, что это тестовые данные
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [ ]:


# Функция для вычисления Pixel Accuracy
def pixel_accuracy(predicted, ground_truth, threshold=0.1):
    """
    Вычисляет точность пикселей: насколько предсказания близки к истинным.
    """
    diff = np.abs(predicted - ground_truth)
    correct = np.sum(diff < threshold)
    total = np.prod(ground_truth.shape)  # Общее количество пикселей
    return correct / total

# Функция для вычисления MSE
def mean_squared_error(predicted, ground_truth):
    """
    Вычисляет среднеквадратическую ошибку (MSE) между предсказанными и истинными значениями.
    """
    return np.mean((predicted - ground_truth) ** 2)

# Функция для тестирования модели


# Функция для тестирования модели с визуализацией трех изображений
def test_model_with_visualization(model, test_loader, device):
    model.eval()  # Переводим модель в режим оценки (без обучения)

    pixel_acc_list = []
    ssim_list = []
    psnr_list = []
    mse_list = []

    with torch.no_grad():  # Не считаем градиенты, чтобы сэкономить память
        for L, ab in test_loader:
            L, ab = L.to(device), ab.to(device)

            # Предсказание ab-каналов
            output_ab = model(L)

            # Преобразуем тензоры в numpy массивы для метрик
            output_ab = output_ab.squeeze(0).cpu().numpy() * 128  # Преобразуем в диапазон от -128 до 128
            ab_np = ab.squeeze(0).cpu().numpy() * 128  # Истинные значения для сравнения


            single_output_ab = output_ab[0]  # Берём первое изображение из батча

            single_ab_np = ab_np[0]

            single_L = L[0]

            # Переводим аб-каналы в (32, 32, 2) для оценки
            ab_pred = single_output_ab.transpose(1, 2, 0)
            ab_true = single_ab_np.transpose(1, 2, 0)

            # Визуализируем результаты (три изображения)
            if len(pixel_acc_list) == 0:  # Визуализируем только первый пример
                L_np = single_L.squeeze(0).cpu().numpy() * 100  # Л-канал изображения

                # Собираем обратно LAB картинку для предсказанного изображения
                lab_img_pred = np.zeros((32, 32, 3), dtype=np.float32)
                lab_img_pred[:, :, 0] = L_np
                lab_img_pred[:, :, 1:] = ab_pred
                rgb_img_pred = lab2rgb(lab_img_pred)

                # Собираем обратно LAB картинку для истинного изображения
                lab_img_true = np.zeros((32, 32, 3), dtype=np.float32)
                lab_img_true[:, :, 0] = L_np
                lab_img_true[:, :, 1:] = ab_true
                rgb_img_true = lab2rgb(lab_img_true)

                # Визуализируем три изображения
                plt.figure(figsize=(10, 5))

                # Оригинальное изображение (в цвете)
                plt.subplot(1, 3, 1)
                plt.imshow(lab2rgb(np.concatenate([L_np[..., np.newaxis], ab_true], axis=-1)))
                plt.title("Original Image (RGB)")
                plt.axis('off')

                # Серая версия (L-канал)
                plt.subplot(1, 3, 2)
                plt.imshow(L_np, cmap='gray')
                plt.title("Gray Version (L-channel)")
                plt.axis('off')

                # Раскрашенная версия (предсказание модели)
                plt.subplot(1, 3, 3)
                plt.imshow(rgb_img_pred)
                plt.title("Model Prediction (Colored)")
                plt.axis('off')

                plt.show()

            # Оценка Pixel Accuracy
            pixel_acc = pixel_accuracy(ab_pred, ab_true)
            pixel_acc_list.append(pixel_acc)

            # Оценка SSIM
            ssim_value = ssim(lab_img_pred, lab_img_true, multichannel=True, win_size=3, data_range=128)

            ssim_list.append(ssim_value)

            # Оценка PSNR
            psnr_value = psnr(ab_true, ab_pred, data_range=128)
            psnr_list.append(psnr_value)

            # Оценка MSE
            mse_value = mean_squared_error(ab_pred, ab_true)
            mse_list.append(mse_value)

    # Среднее значение метрик
    avg_pixel_acc = np.mean(pixel_acc_list)
    avg_ssim = np.mean(ssim_list)
    avg_psnr = np.mean(psnr_list)
    avg_mse = np.mean(mse_list)

    return avg_pixel_acc, avg_ssim, avg_psnr, avg_mse

# Пример использования
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# test_loader - это DataLoader для тестового набора данных

pixel_acc, ssim_val, psnr_val, mse_val = test_model_with_visualization(model, test_loader, device)

print(f"Pixel Accuracy: {pixel_acc:.4f}")
print(f"SSIM: {ssim_val:.4f}")
print(f"PSNR: {psnr_val:.4f}")
print(f"MSE: {mse_val:.4f}")


In [ ]:


# Загрузка изображения с интернета
image_url = "https://www.1zoom.me/big2/819/321881-svetik.jpg"  # Замените на актуальную ссылку
response = requests.get(image_url, stream=True)
img = Image.open(response.raw)

# Предобработка изображения
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Размер, соответствующий входу модели
    transforms.ToTensor(),  # Перевод в тензор
])

img_tensor = transform(img).unsqueeze(0)  # Добавляем дополнительную ось для батча (1, C, H, W)

# Преобразуем в цветовое пространство Lab
img_rgb = img_tensor.squeeze().numpy().transpose(1, 2, 0)  # Преобразуем обратно в numpy array
img_lab = color.rgb2lab(img_rgb)  # Преобразуем в Lab

L = img_lab[:, :, 0] / 100.0  # Канал L (светлота)
ab = img_lab[:, :, 1:] / 128.0  # Каналы a и b

# Перемещаем данные на нужное устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Преобразуем данные в тензоры
L_tensor = torch.tensor(L, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)  # (1, 1, H, W)
ab_tensor = torch.tensor(ab, dtype=torch.float32).unsqueeze(0).permute(0, 3, 1, 2).to(device)  # (1, 2, H, W)

# Перемещаем модель на тот же девайс
model.to(device)
model.eval()  # Переводим модель в режим оценки

# Прогоняем изображение через модель
with torch.no_grad():  # Отключаем вычисление градиентов
    output = model(L_tensor)  # Получаем предсказание модели

# Визуализация
output = output.squeeze().permute(1, 2, 0).cpu().numpy()  # Переводим результат в numpy для визуализации

# Собираем Lab и конвертируем в RGB
ab_pred = output * 128.0  # Каналы a, b в диапазоне [-128, 128]
L_pred = L * 100.0  # Канал L в диапазоне [0, 100]

lab_pred = np.concatenate([L_pred[..., np.newaxis], ab_pred], axis=-1)
rgb_pred = color.lab2rgb(lab_pred)

# Визуализируем
plt.figure(figsize=(12, 6))

# Оригинал
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title("Оригинальное изображение")

# Результат модели
plt.subplot(1, 2, 2)
plt.imshow(rgb_pred)
plt.title("Предсказание модели")

plt.show()
